# Preparing data for collaboration network
Here we consider users v2.  
In this notebook, we try to create multiple collaboration graphs separated by time. The time is separated by month by looking up their commit overlapping time.

# Only consider users v2 that have commits

In [1]:
import pandas as pd

In [22]:
users_df = pd.read_csv('../dataset/v2/users.csv')
display(users_df.head())
print(users_df.shape)

,Unnamed: 0,id,login,username,company,bio,location,email,followers,following,repos,starred_repos
0,0,MDQ6VXNlcjgxMDQzOA==,gaearon,dan,@facebook,NaN,NaN,dan.abramov@gmail.com,"['jayden-n', 'arguetaj26', 'kirillklimovets', ...","['kbuzzard', 'sebmarkbage', 'rauchg', 'lukewes...","['facebook/react', 'facebook/react-native', 'r...","[{'name': 'rauchg/how-is-this-not-illegal', 's..."
1,1,MDQ6VXNlcjg1NDc1Mzg=,buckyroberts,Bucky Roberts,thenewboston,linkedin.com/in/buckyroberts,"New York, NY",NaN,"['kimjoonja', 'mayilairajesh', 'ckehelly', 'pa...","['MaxLuxon', 'nidhi99verma', 'yashtazor', 'Sud...","['thenewboston-developers/Website', 'thenewbos...","[{'name': 'thenewboston-developers/tnbOS', 'st..."
2,2,MDQ6VXNlcjIyODEwODg=,sdras,Sarah Drasner,Netlify,comprehension over configuration,All over the place,sarah.drasner@gmail.com,"['spandalai26', 'wangliuyi09', 'RakibulHSium',...","['segunadebayo', 'concavelenz', 'robb0wen', 'n...",[],"[{'name': 'danielroe/nuxt-zero-js', 'starred_a..."
3,3,MDQ6VXNlcjU1NTA4NTA=,bradtraversy,Brad Traversy,Traversy Media,Full stack web developer and online instructor...,Massachusetts,traversymedia@gmail.com,"['wanwei9025', 'wangliuyi09', 'premsagar2000',...","['Alaev', 'toddmotto', 'taylorotwell', 'getify...",[],[{'name': 'bushblade/TraversyMediaScrollActive...
4,4,MDQ6VXNlcjUwMDM5MDM=,StephenGrider,Stephen Grider,NaN,NaN,NaN,NaN,"['rafaelnacle', 'alululululuer', 'soleira', 'j...","['dthms', 'JosephSmith127', 'bibanul', 'pieter...",[],"[{'name': 'redis/node-redis', 'starred_at': '2..."


(34678, 12)


In [23]:
commits = pd.read_csv('../dataset/data/commits.csv')
display(commits.head())
print(commits.shape)

,Unnamed: 0,user,repo,commits
0,0,gaearon,facebook/react,"['2023-03-30T21:05:03', '2023-03-25T00:05:23',..."
1,1,gaearon,facebook/react-native,"['2020-07-10T20:24:20', '2020-03-26T20:59:39',..."
2,2,gaearon,reactjs/react.dev,"['2023-05-03T22:07:50', '2023-05-03T21:18:00',..."
3,3,gaearon,reactjs/rfcs,"['2022-03-28T18:13:52', '2022-03-28T18:11:33',..."
4,4,gaearon,codesandbox/sandpack,"['2023-04-09T22:06:53', '2022-05-25T09:13:12',..."


(36411, 4)


In [25]:
users = users_df['login'].unique()
print(f"unique users in users v2: {len(users)}")
commit_users = commits['user'].unique()
print(f"unique users in commits: {len(commit_users)}")
    
# try see if all users have commits
count = 0
for user in users:
    if user not in commit_users:
        count += 1
print(f"count of users not in commits: {count}")

unique users in users v2: 34678
unique users in commits: 5393
count of users not in commits: 29285


## Reduce users size by considering only users that have commits

In [26]:
reduced_users_df = users_df[users_df['login'].isin(commit_users)]
print(reduced_users_df.shape)

(5393, 12)


In [27]:
# save to csv
reduced_users_df.to_csv('../dataset/postprocess-data/reduced_users.csv', index=False)

## Construct collaboration graph by month

### Create a Repo_Month_User dictionary

In [28]:
# load reduced users
reduced_users_df = pd.read_csv('../dataset/postprocess-data/reduced_users.csv')
display(reduced_users_df.head())
print(reduced_users_df.shape)

,Unnamed: 0,id,login,username,company,bio,location,email,followers,following,repos,starred_repos
0,0,MDQ6VXNlcjgxMDQzOA==,gaearon,dan,@facebook,NaN,NaN,dan.abramov@gmail.com,"['jayden-n', 'arguetaj26', 'kirillklimovets', ...","['kbuzzard', 'sebmarkbage', 'rauchg', 'lukewes...","['facebook/react', 'facebook/react-native', 'r...","[{'name': 'rauchg/how-is-this-not-illegal', 's..."
1,1,MDQ6VXNlcjg1NDc1Mzg=,buckyroberts,Bucky Roberts,thenewboston,linkedin.com/in/buckyroberts,"New York, NY",NaN,"['kimjoonja', 'mayilairajesh', 'ckehelly', 'pa...","['MaxLuxon', 'nidhi99verma', 'yashtazor', 'Sud...","['thenewboston-developers/Website', 'thenewbos...","[{'name': 'thenewboston-developers/tnbOS', 'st..."
2,5,MDQ6VXNlcjM5NzE3MDk5,WebDevSimplified,NaN,WebDevSimplified,NaN,Nebraska,kyle@webdevsimplified.com,"['th3mon', 'kencargill', 'Paarsah', 'Leandrome...",[],['colinhacks/zod'],[]
3,7,MDQ6VXNlcjI0MTEzOA==,karpathy,Andrej,NaN,I like to train Deep Neural Nets on large data...,Stanford,andrej.karpathy@gmail.com,"['tvldz', 'duck-bongos', 'jecky100000', 'vvust...","['ErikBjare', 'ahojnnes', 'petewarden', 'mniel...",['huggingface/transformers'],"[{'name': 'LAION-AI/Open-Assistant', 'starred_..."
4,8,MDQ6VXNlcjIyNTQ3MzE=,diego3g,Diego Fernandes,@Rocketseat,CTO at @Rocketseat. Passionate about education...,Brasil,diego.schell.f@gmail.com,"['mutheusalmeida', 'rehfeld13', 'andressaviana...","['mpedroni', 'RennanD', 'murilosf1', 'jaksonxa...","['maykbrito/mini-video-me', 'Rocketseat/eslint...","[{'name': 'partykit/partykit', 'starred_at': '..."


(5393, 12)


In [29]:
# load commits
commits = pd.read_csv('../dataset/data/commits.csv')
display(commits.head())
print(commits.shape)

,Unnamed: 0,user,repo,commits
0,0,gaearon,facebook/react,"['2023-03-30T21:05:03', '2023-03-25T00:05:23',..."
1,1,gaearon,facebook/react-native,"['2020-07-10T20:24:20', '2020-03-26T20:59:39',..."
2,2,gaearon,reactjs/react.dev,"['2023-05-03T22:07:50', '2023-05-03T21:18:00',..."
3,3,gaearon,reactjs/rfcs,"['2022-03-28T18:13:52', '2022-03-28T18:11:33',..."
4,4,gaearon,codesandbox/sandpack,"['2023-04-09T22:06:53', '2022-05-25T09:13:12',..."


(36411, 4)


In [42]:
import ast
# Convert a string representation of list into list
commits['commits_list'] = commits['commits'].apply(lambda x: ast.literal_eval(x))

In [51]:
print(commits.head())

   Unnamed: 0     user                   repo   
0           0  gaearon         facebook/react  \
1           1  gaearon  facebook/react-native   
2           2  gaearon      reactjs/react.dev   
3           3  gaearon           reactjs/rfcs   
4           4  gaearon   codesandbox/sandpack   

                                             commits   
0  ['2023-03-30T21:05:03', '2023-03-25T00:05:23',...  \
1  ['2020-07-10T20:24:20', '2020-03-26T20:59:39',...   
2  ['2023-05-03T22:07:50', '2023-05-03T21:18:00',...   
3  ['2022-03-28T18:13:52', '2022-03-28T18:11:33',...   
4  ['2023-04-09T22:06:53', '2022-05-25T09:13:12',...   

                                        commits_list  
0  [2023-03-30T21:05:03, 2023-03-25T00:05:23, 202...  
1  [2020-07-10T20:24:20, 2020-03-26T20:59:39, 202...  
2  [2023-05-03T22:07:50, 2023-05-03T21:18:00, 202...  
3  [2022-03-28T18:13:52, 2022-03-28T18:11:33, 202...  
4  [2023-04-09T22:06:53, 2022-05-25T09:13:12, 202...  


In [69]:
# use pd.date_range to create a range of dates starting from 2013-05 to 2023-04, with a frequency of 1 month
date_range = pd.date_range(start='2013-03', end='2023-06', freq='M')
# convert the date_range into a list
date_range = date_range.tolist()
# convert the list of dates into a list of strings
date_range = [str(date) for date in date_range]
# only use first 10 characters of the date string as the month indicator
date_range = [date[:7] for date in date_range]
print(date_range)
print(len(date_range))

['2013-03', '2013-04', '2013-05', '2013-06', '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12', '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06', '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12', '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06', '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12', '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06', '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12', '2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09'

In [73]:
repo_month_users = {}
for repo in commits['repo'].unique():
    repo_month_users[repo] = {}
    for date in date_range:
        repo_month_users[repo][date] = set()

# iterate through each row of the commits dataframe and append the user to the corresponding repo and month
for index, row in commits.iterrows():
    repo = row['repo']
    user = row['user']
    commits_date_list = row['commits_list']
    for commit_date in commits_date_list:
        date = commit_date[:7]
        if date in date_range:
            repo_month_users[repo][date].add(user)
        

In [76]:
# check some of the values
print(repo_month_users['facebook/react']['2023-03'])
print(repo_month_users['facebook/react-native']['2022-08'])

{'sebmarkbage', 'kassens', 'gaearon', 'sammy-SC', 'tyao1', 'markerikson', 'josephsavona', 'sophiebits', 'Jarred-Sumner', 'poteto', 'rickhanlonii', 'eps1lon', 'acdlite'}
{'mdvacca', 'chiaramooney', 'JoshuaGross', 'hramos', 'tido64', 'yungsters', 'neildhar', 'EvanBacon', 'SamChou19815', 'sammy-SC', 'NickGerleman', 'cortinico', 'kelset', 'Kudo', 'janicduplessis', 'huntie', 'javache', 'tyao1', 'thymikee', 'RSNara'}


In [77]:
# save this dict to pickle
import pickle
with open('../dataset/postprocess-data/repo_month_users.pickle', 'wb') as handle:
    pickle.dump(repo_month_users, handle, protocol=pickle.HIGHEST_PROTOCOL)


### Construct collaboration relationship based on Repo_Month_User dictionary